In [1]:
import random

import evaluate
import numpy as np
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    BertConfig,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)

from model.bert_configs import shadow, target

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [2]:
dataset = load_dataset("yelp_review_full", split=["train+test"])[0].train_test_split(
    test_size=0.5, stratify_by_column="label"
)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [3]:
tokenized_dataset = dataset.map(
    lambda x: tokenizer(
        x["text"], return_tensors="pt", padding=True, max_length=512, truncation=True
    ),
    batched=True,
)
tokenized_dataset.set_format(type="torch")

In [4]:
target_subset = tokenized_dataset["train"].train_test_split(
    test_size=0.5, stratify_by_column="label"
)

In [5]:
N_SHADOWS = 10

shadow_subsets = []
for i in range(N_SHADOWS):
    shadow_subsets.append(
        tokenized_dataset["test"].train_test_split(
            test_size=0.5, stratify_by_column="label"
        )
    )

In [6]:
def compute_metrics(eval_preds):
    accuracy = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# Train Target

In [7]:
target_bert_config = BertConfig(**target)
target_classifier = BertForSequenceClassification(config=target_bert_config)

In [8]:
training_args = TrainingArguments(
    output_dir="saved_models/target",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
)

trainer = Trainer(
    model=target_classifier,
    args=training_args,
    train_dataset=target_subset["train"],
    eval_dataset=target_subset["test"],
    compute_metrics=compute_metrics,
)

In [9]:
trainer.train()

  0%|          | 0/27350 [00:00<?, ?it/s]

{'loss': 1.3388, 'grad_norm': 8.534919738769531, 'learning_rate': 4.9085923217550275e-05, 'epoch': 0.18}
{'loss': 1.0463, 'grad_norm': 18.86072540283203, 'learning_rate': 4.817184643510055e-05, 'epoch': 0.37}
{'loss': 1.0083, 'grad_norm': 6.08780574798584, 'learning_rate': 4.725776965265082e-05, 'epoch': 0.55}
{'loss': 0.9996, 'grad_norm': 5.470260143280029, 'learning_rate': 4.63436928702011e-05, 'epoch': 0.73}
{'loss': 0.973, 'grad_norm': 11.012569427490234, 'learning_rate': 4.542961608775137e-05, 'epoch': 0.91}


  0%|          | 0/2735 [00:00<?, ?it/s]

{'eval_loss': 0.9602767825126648, 'eval_accuracy': 0.5747885714285714, 'eval_runtime': 543.8096, 'eval_samples_per_second': 321.804, 'eval_steps_per_second': 5.029, 'epoch': 1.0}
{'loss': 0.9444, 'grad_norm': 8.25650405883789, 'learning_rate': 4.4515539305301646e-05, 'epoch': 1.1}
{'loss': 0.9317, 'grad_norm': 8.17536735534668, 'learning_rate': 4.360146252285192e-05, 'epoch': 1.28}
{'loss': 0.918, 'grad_norm': 17.78175163269043, 'learning_rate': 4.26873857404022e-05, 'epoch': 1.46}
{'loss': 0.9209, 'grad_norm': 11.881813049316406, 'learning_rate': 4.1773308957952465e-05, 'epoch': 1.65}
{'loss': 0.9128, 'grad_norm': 9.233500480651855, 'learning_rate': 4.0859232175502744e-05, 'epoch': 1.83}


  0%|          | 0/2735 [00:00<?, ?it/s]

...

  0%|          | 0/2735 [00:00<?, ?it/s]

{'eval_loss': 1.204210638999939, 'eval_accuracy': 0.5686171428571428, 'eval_runtime': 505.7598, 'eval_samples_per_second': 346.014, 'eval_steps_per_second': 5.408, 'epoch': 9.0}
{'loss': 0.6043, 'grad_norm': 20.543245315551758, 'learning_rate': 4.296160877513712e-06, 'epoch': 9.14}
{'loss': 0.6007, 'grad_norm': 8.099677085876465, 'learning_rate': 3.382084095063986e-06, 'epoch': 9.32}
{'loss': 0.5981, 'grad_norm': 7.13821268081665, 'learning_rate': 2.4680073126142596e-06, 'epoch': 9.51}
{'loss': 0.611, 'grad_norm': 14.220907211303711, 'learning_rate': 1.5539305301645339e-06, 'epoch': 9.69}
{'loss': 0.6106, 'grad_norm': 8.447056770324707, 'learning_rate': 6.398537477148081e-07, 'epoch': 9.87}


  0%|          | 0/2735 [00:00<?, ?it/s]

{'eval_loss': 1.2226758003234863, 'eval_accuracy': 0.5682971428571428, 'eval_runtime': 505.5062, 'eval_samples_per_second': 346.188, 'eval_steps_per_second': 5.41, 'epoch': 10.0}
{'train_runtime': 21638.9737, 'train_samples_per_second': 80.873, 'train_steps_per_second': 1.264, 'train_loss': 0.790011998373586, 'epoch': 10.0}


TrainOutput(global_step=27350, training_loss=0.790011998373586, metrics={'train_runtime': 21638.9737, 'train_samples_per_second': 80.873, 'train_steps_per_second': 1.264, 'total_flos': 6.9220198656e+16, 'train_loss': 0.790011998373586, 'epoch': 10.0})

# Train Shadows

In [10]:
shadow_bert_config = BertConfig(**shadow)
for i in range(N_SHADOWS):
    shadow_classifier = BertForSequenceClassification(config=shadow_bert_config)
    training_args = TrainingArguments(
        output_dir=f"saved_models/shadow_{i}",
        overwrite_output_dir=True,
        num_train_epochs=10,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        eval_strategy="epoch",
        save_strategy="epoch",
        report_to="none",
    )
    trainer = Trainer(
        model=shadow_classifier,
        args=training_args,
        train_dataset=shadow_subsets[i]["train"],
        eval_dataset=shadow_subsets[i]["test"],
        compute_metrics=compute_metrics,
    )
    trainer.train()

  0%|          | 0/27350 [00:00<?, ?it/s]

{'loss': 1.4068, 'grad_norm': 6.6257758140563965, 'learning_rate': 4.9085923217550275e-05, 'epoch': 0.18}
{'loss': 1.1097, 'grad_norm': 12.019930839538574, 'learning_rate': 4.817184643510055e-05, 'epoch': 0.37}
{'loss': 1.0351, 'grad_norm': 6.489343166351318, 'learning_rate': 4.725776965265082e-05, 'epoch': 0.55}
{'loss': 0.9967, 'grad_norm': 12.220290184020996, 'learning_rate': 4.63436928702011e-05, 'epoch': 0.73}
{'loss': 0.9778, 'grad_norm': 3.00787615776062, 'learning_rate': 4.542961608775137e-05, 'epoch': 0.91}


  0%|          | 0/2735 [00:00<?, ?it/s]

{'eval_loss': 0.9851199984550476, 'eval_accuracy': 0.5673714285714285, 'eval_runtime': 184.4302, 'eval_samples_per_second': 948.868, 'eval_steps_per_second': 14.829, 'epoch': 1.0}
{'loss': 0.9617, 'grad_norm': 6.667905807495117, 'learning_rate': 4.4515539305301646e-05, 'epoch': 1.1}
{'loss': 0.9375, 'grad_norm': 15.998618125915527, 'learning_rate': 4.360146252285192e-05, 'epoch': 1.28}
{'loss': 0.9276, 'grad_norm': 15.357975006103516, 'learning_rate': 4.26873857404022e-05, 'epoch': 1.46}
{'loss': 0.9392, 'grad_norm': 8.30373477935791, 'learning_rate': 4.1773308957952465e-05, 'epoch': 1.65}
{'loss': 0.9349, 'grad_norm': 3.7897067070007324, 'learning_rate': 4.0859232175502744e-05, 'epoch': 1.83}


  0%|          | 0/2735 [00:00<?, ?it/s]

...

{'eval_loss': 0.9966398477554321, 'eval_accuracy': 0.58216, 'eval_runtime': 185.1505, 'eval_samples_per_second': 945.177, 'eval_steps_per_second': 14.772, 'epoch': 8.0}
{'loss': 0.7759, 'grad_norm': 12.02391242980957, 'learning_rate': 9.780621572212066e-06, 'epoch': 8.04}
{'loss': 0.7655, 'grad_norm': 17.789291381835938, 'learning_rate': 8.86654478976234e-06, 'epoch': 8.23}
{'loss': 0.7682, 'grad_norm': 5.542922496795654, 'learning_rate': 7.952468007312615e-06, 'epoch': 8.41}
{'loss': 0.762, 'grad_norm': 5.86365270614624, 'learning_rate': 7.038391224862889e-06, 'epoch': 8.59}
{'loss': 0.7722, 'grad_norm': 15.101332664489746, 'learning_rate': 6.124314442413163e-06, 'epoch': 8.78}
{'loss': 0.7744, 'grad_norm': 6.7013325691223145, 'learning_rate': 5.210237659963437e-06, 'epoch': 8.96}


  0%|          | 0/2735 [00:00<?, ?it/s]

{'eval_loss': 1.0112935304641724, 'eval_accuracy': 0.5801142857142857, 'eval_runtime': 184.9767, 'eval_samples_per_second': 946.065, 'eval_steps_per_second': 14.786, 'epoch': 9.0}
{'loss': 0.7625, 'grad_norm': 7.166996479034424, 'learning_rate': 4.296160877513712e-06, 'epoch': 9.14}
{'loss': 0.7485, 'grad_norm': 17.034269332885742, 'learning_rate': 3.382084095063986e-06, 'epoch': 9.32}
{'loss': 0.7561, 'grad_norm': 5.779067039489746, 'learning_rate': 2.4680073126142596e-06, 'epoch': 9.51}
{'loss': 0.7615, 'grad_norm': 7.638485431671143, 'learning_rate': 1.5539305301645339e-06, 'epoch': 9.69}
{'loss': 0.7501, 'grad_norm': 3.955409049987793, 'learning_rate': 6.398537477148081e-07, 'epoch': 9.87}


  0%|          | 0/2735 [00:00<?, ?it/s]

{'eval_loss': 1.0295977592468262, 'eval_accuracy': 0.5794571428571429, 'eval_runtime': 198.1825, 'eval_samples_per_second': 883.024, 'eval_steps_per_second': 13.8, 'epoch': 10.0}
{'train_runtime': 8117.0759, 'train_samples_per_second': 215.595, 'train_steps_per_second': 3.369, 'train_loss': 0.8565058957506142, 'epoch': 10.0}


# Train Attacker

## Dataset Creation

In [ ]:
from transformers import AutoModelForSequenceClassification
from torch.nn import functional as F
from torch.utils.data import DataLoader

attack_dataset_X = []  # shadow model predicition scores
attack_dataset_y = []  # membership / non-membership

for i in range(10):
    shadow_model = AutoModelForSequenceClassification.from_pretrained(
        f"saved_models/shadow_{i}"
    )
    shadow_train_dl = DataLoader(shadow_subsets[i]["train"], batch_size=100)
    shadow_test_dl = DataLoader(shadow_subsets[i]["test"], batch_size=100)
    shadow_model.eval()
    for batch in shadow_train_dl:
        with torch.inference_mode():
            logits = shadow_model(**batch)["logits"]
        probs = F.softmax(logits, dim=-1).cpu().numpy()
        attack_dataset_X.append(probs)
        attack_dataset_y.append(np.ones(len(probs)))
    for batch in shadow_test_dl:
        with torch.inference_mode():
            logits = shadow_model(**batch)["logits"]
        probs = F.softmax(logits, dim=-1).cpu().numpy()
        attack_dataset_X.append(probs)
        attack_dataset_y.append(np.zeros(len(probs)))

attack_X_train = np.hstack(attack_dataset_X)
attack_y_train = np.hstack(attack_dataset_y)

np.savez_compressed(
    "saved_datasets/attack_train_dataset",
    attack_X_train=attack_X_train,
    attack_y_train=attack_y_train,
)

## Training

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

base_xgb_attacker = XGBClassifier(objective="binary:logistic", eval_metric="auc")
gridsearch_clf = GridSearchCV(
    base_xgb_attacker,
    {
        "max_depth": [1, 2, 3, 4, 5],
        "n_estimators": [2, 5, 10, 50]
    },
    verbose=1
)

gridsearch_clf.fit(attack_X_train, attack_y_train)
best_xgb_attacker = gridsearch_clf.best_estimator_
best_xgb_attacker.save_model("saved_models/attacker/XGB_attacker.json")

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(attack_y_train, best_xgb_attacker.predict_proba(attack_X_train)[:, 1])

# Inference

## Test Dataset Creation

In [ ]:
from transformers import AutoModelForSequenceClassification
from torch.nn import functional as F
from torch.utils.data import DataLoader

attack_dataset_X = []  # target model predicition scores
attack_dataset_y = []  # membership / non-membership

target_model = AutoModelForSequenceClassification.from_pretrained(
    "saved_models/target"
)
target_train_dl = DataLoader(target_subset["train"], batch_size=100)
target_test_dl = DataLoader(target_subset["test"], batch_size=100)
target_model.eval()
for batch in target_train_dl:
    with torch.inference_mode():
        logits = target_model(**batch)["logits"]
    probs = F.softmax(logits, dim=-1).cpu().numpy()
    attack_dataset_X.append(probs)
    attack_dataset_y.append(np.ones(len(probs)))
for batch in target_test_dl:
    with torch.inference_mode():
        logits = target_model(**batch)["logits"]
    probs = F.softmax(logits, dim=-1).cpu().numpy()
    attack_dataset_X.append(probs)
    attack_dataset_y.append(np.zeros(len(probs)))

attack_X_test = np.hstack(attack_dataset_X)
attack_y_test = np.hstack(attack_dataset_y)

np.savez_compressed(
    "saved_datasets/attack_test_dataset",
    attack_X_test=attack_X_test,
    attack_y_test=attack_y_test,
)

## Run Attack